# Worksheet 08

Name: Chao-Jen, Chiu 

UID: U29089552
 

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [19]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

c1 = np.random.normal(mean, stdev, 10).tolist()
print(c1)

[5.77047752025307, 6.433918801717679, 6.092533869375476, 4.93631955278417, 6.215279366258548, 5.4456393874708375, 4.079128273498911, 3.864734090188887, 7.075818740006536, 4.777801628236898]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [20]:
c2 = np.random.normal( 8, 1, 10 ).tolist()
print(c2)

[7.383724607407848, 7.8346621313375024, 7.272320503402302, 7.126517344140005, 10.068107519602272, 7.335349778667307, 7.586332030574523, 9.06918512209086, 6.387265527685649, 8.033778759804315]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [21]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = c1.pop()
        data.append(p1)
    else:
        p2 = c2.pop()
        data.append(p2)
print(data)

[8.033778759804315, 6.387265527685649, 9.06918512209086, 7.586332030574523, 4.777801628236898, 7.075818740006536, 3.864734090188887, 4.079128273498911, 7.335349778667307, 10.068107519602272]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.


The parameters of a Gaussian Mixture Model (GMM) with 2 mixture components are:

- Means: The mean of each Gaussian component.
- Covariances: The covariance of each Gaussian component.
- Weights: The weight of each Gaussian component.

The values of these parameters that we know are:

- Weights: The weights of the two Gaussian components are equal, since the data is generated from a 50/50 mixture of the two components.

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(C_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(C_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [22]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

c1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(c1)
c2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(c2)

prob_c = [ len(c1) / (len(c1) + len(c2)) , len(c2)/len(c1)+len(c2) ]
mean = [ sum(c1)/len(c1) , sum(c2)/len(c2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1) ,sum(map(lambda x : (x - mean[1])**2, c2)) / len(c2) ]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[8.033778759804315, 6.387265527685649, 9.06918512209086, 7.586332030574523, 7.075818740006536, 7.335349778667307, 10.068107519602272]
[4.777801628236898, 3.864734090188887, 4.079128273498911]
P(C_1) = 0.7,  P(C_2) = 3.4285714285714284
mean_1 = 7.936548211204495,  mean_2 = 4.2405546639748986
var_1 = 1.351584031600104,  var_2 = 0.1519779612772291


/Users/cjchiu99/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


e) For each data point, compute `P(C_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [23]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append(pdf_i[0] * prob_c[0] / prob_x )
    prob_c1_x.append(pdf_i[1] * prob_c[1] / prob_x )

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


point =  8.033778759804315
probability of observing that point if it came from cluster 0 =  0.2944036929389428
probability of observing that point if it came from cluster 1 =  1.4010424775363007e-135
point =  6.387265527685649
probability of observing that point if it came from cluster 0 =  0.15302020100457603
probability of observing that point if it came from cluster 1 =  1.2415592855115653e-43
point =  9.06918512209086
probability of observing that point if it came from cluster 0 =  0.20776582393412574
probability of observing that point if it came from cluster 1 =  1.6553472551886623e-219
point =  7.586332030574523
probability of observing that point if it came from cluster 0 =  0.28542208936876273
probability of observing that point if it came from cluster 1 =  1.5052084059385658e-105
point =  4.777801628236898
probability of observing that point if it came from cluster 0 =  0.019232301513937518
probability of observing that point if it came from cluster 1 =  0.0050764424079795265

f) Having computed `P(C_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(C_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [24]:
prob_c = [sum(prob_c0_x)/ len(prob_c0_x), sum(prob_c1_x)/ len(prob_c1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), 
        sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x) ]
var = [sum([x[0] * (x[1] - mean[0]) ** 2 for x in zip(prob_c0_x, data)]) / sum(prob_c0_x),
       sum([x[0] * (x[1] - mean[1]) ** 2 for x in zip(prob_c1_x, data)]) / sum(prob_c1_x)]


print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(C_1) = 0.7442024599636075,  P(C_2) = 0.25579754003639243
mean_1 = 7.748230043924343,  mean_2 = 4.1497595630988835
var_1 = 1.8335361537589874,  var_2 = 0.1204895915431079


g) Update `P(C_j | X_i)`. Comment on any differences or lack thereof you observe.

In [25]:
prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append(pdf_i[0] * prob_c[0] / prob_x )
    prob_c1_x.append(pdf_i[1] * prob_c[1] / prob_x )

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()

point =  8.033778759804315
probability of observing that point if it came from cluster 0 =  0.2149581530535458
probability of observing that point if it came from cluster 1 =  7.567903158352416e-226
point =  6.387265527685649
probability of observing that point if it came from cluster 0 =  0.16518957565679182
probability of observing that point if it came from cluster 1 =  4.3339924937031686e-75
point =  9.06918512209086
probability of observing that point if it came from cluster 0 =  0.16784694488433113
probability of observing that point if it came from cluster 1 =  0.0
point =  7.586332030574523
probability of observing that point if it came from cluster 0 =  0.2167342675219598
probability of observing that point if it came from cluster 1 =  7.463823917891882e-177
point =  4.777801628236898
probability of observing that point if it came from cluster 0 =  0.05857348379411106
probability of observing that point if it came from cluster 1 =  4.170831510186962e-06
point =  7.075818740006

h) Use `P(C_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [26]:
assignments = [[], []]
for x in zip(data, prob_c0_x):
    p = x[1]
    d = x[0]
    if random.random() <= p:
        assignments[0].append(d)
    else:
        assignments[1].append(d)
print(f"points in cluster 0 : {assignments[0]}\npoints in cluster 1 : {assignments[1]}")

points in cluster 0 : [8.033778759804315, 6.387265527685649, 9.06918512209086, 7.586332030574523, 4.777801628236898, 7.075818740006536, 7.335349778667307, 10.068107519602272]
points in cluster 1 : [3.864734090188887, 4.079128273498911]
